Refer to the lectures, implement Bayesian classifier from scratch, use it on MNIST dataset, and test.

Generate the confusion matrix and F1 score for each Dist.

### discret case


In [ ]:
import numpy as np
from keras.datasets import mnist
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
#rearrange the data 
def arder_x_y(x,y):
  uniq=np.unique(y)
  x_ordered=[x[np.where(y==i)] for i in uniq]
  t=np.sort(y)
  return x_ordered,t

In [ ]:
#probabilityies of y
def prob_calc(y):
  p= [ len(y[y==i])/len(y)  for i in np.unique(y)]
  return p

In [ ]:
# probability of y given x for all classes and features
def prob_x_given_y(x,y,x_uniq):
  l=np.empty((0,len(x_uniq)))
  for xx in x:
    for x_ in np.transpose(xx):
      r=[len(x_[x_==uni])/len(x_) for uni in x_uniq]
      l=np.vstack([l,np.array(r)])
  l=l.reshape(len(np.unique(y)),-1,len(np.unique(x_uniq)))
  return np.asarray(l)

In [ ]:
#calculate probability of certain x given certain y
def get_x_given_y(l,given_y,given_x,x_uniq):
  u=[ l[given_y,idx,ua]   for idx, ua in enumerate(given_x)]
  u=np.array(u)
  return u

In [ ]:
#the total peobability of x 
#add 1e-6 to avoid log(0)
def get_probs(p,y,u):
  pb=[np.log2(pp+1e-6) for pp in p] 
  y_unique=np.unique(y)
  for y_u in y_unique:
    for prob in np.array(u[y_u]):
      pb[y_u]=pb[y_u]+np.log2(prob+1e-6)
  return pb

Bayesian classifier for discrete values on minist

In [ ]:
#read the data
(train_X, train_y), (test_X, test_y) = mnist.load_data()
#reshape the data
train_X=train_X.reshape([60000,28*28])
test_X=test_X.reshape([10000,28*28])
#reorder the data
x,y=arder_x_y(train_X,train_y)
#probability of y
p=prob_calc(y)
#get unique values of x
x_uniq=np.unique(train_X)
#probability of y given x for all classes and features
l=prob_x_given_y(x,y,x_uniq)

In [ ]:
#get y predicted
y_pred=np.empty([0])
for given_x in test_X:
  u=[get_x_given_y(l,given_y,given_x,x_uniq) for given_y in np.unique(y)]
  pb= get_probs(p,y,u)
  y_pred=np.append(y_pred ,np.argmax(pb))

In [ ]:
f1_score(test_y, y_pred, average='macro')

0.8436903480338364

In [ ]:
confusion_matrix(test_y, y_pred, labels=[0,1,2,3,4,5,6,7,8,9])

array([[ 882,    0,    4,    3,    2,   49,   15,    1,   24,    0],
       [   0, 1064,   21,    5,    1,   12,    5,    0,   27,    0],
       [  16,    5,  859,   41,   19,    5,   25,    9,   52,    1],
       [   2,    1,   34,  854,    0,   30,   10,   14,   39,   26],
       [   3,    0,    7,    0,  805,    7,   17,    0,   30,  113],
       [  24,    1,   11,  112,   20,  650,   19,    8,   28,   19],
       [  12,    6,   26,    1,   12,   37,  854,    0,   10,    0],
       [   1,   14,   13,    4,   26,    2,    1,  847,   38,   82],
       [  15,    3,   15,   66,   10,   36,    7,    7,  785,   30],
       [  11,    4,    4,    9,   71,   11,    1,   22,   24,  852]])

In [ ]:
accuracy_score(test_y, y_pred)

0.8452